In [14]:
from JHProgress import JHProgress
from JHProgress.tsf import TSF
import pandas as pd
import numpy as np
import yaml

ModuleNotFoundError: No module named 'JHProgress.tsf'; 'JHProgress' is not a package

In [17]:
import JHProgress.tsf

ModuleNotFoundError: No module named 'JHProgress.tsf'; 'JHProgress' is not a package

In [10]:
JHProgress.JHProgress()

2025-12-08 12:02:44,229 - INFO - Tickers downloaded successfully


In [2]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Obtencion de datos

In [3]:
jhp=JHProgress()
#jhp.cargar_fundamentales() #Scrapping de la SEC
#jhp.guardar_fundamentales_sql() #Guardar en SQL
prices=jhp.descargar_precios_yahoo(config['Tiempo']['Inicio'],config['Tiempo']['Fin'])
fundamentales=jhp.cargar_fundamentales_sql()
jhp.guardar_precios_fundamentales_sql()

2025-12-08 11:44:43,000 - INFO - Tickers downloaded successfully
c:\Users\palaj\OneDrive\Escritorio\JHProgress\Dev\JHProgress.py:129: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(self.config['Simbolos'], start=start, end=end, progress=False)
2025-12-08 11:44:45,293 - INFO - DataFrame de precios descargado exitosamente
2025-12-08 11:44:59,177 - INFO - DataFrame de fundamentales cargado desde la tabla VW_FUNDAMENTALES_PROCESADOS exitosamente
2025-12-08 11:44:59,419 - INFO - DataFrame de fundamentales y precios mergeado exitosamente para AAPL
2025-12-08 11:44:59,661 - INFO - DataFrame de fundamentales y precios mergeado exitosamente para ANET
2025-12-08 11:44:59,939 - INFO - DataFrame de fundamentales y precios mergeado exitosamente para CORT
2025-12-08 11:45:00,262 - INFO - DataFrame de fundamentales y precios mergeado exitosamente para CPRX
2025-12-08 11:45:00,523 - INFO - DataFrame de fundamentales y precios mergeado exitosamente pa

# Forecasting